In [ ]:
import pandas as pd
import arcpy

In [ ]:
# Load needs data
needs_source = r'C:\Users\daniel.fourquet\Documents\Tasks\VTrans Update\DATA-EXPLORER-UPDATE_MIDTERM-NEEDS\2023_VTrans_MidTerm_Needs_wExtra_Fields.gdb\VTrans_MidTerm_Needs_2023'
needs_fields = ['VDOT_District','CoSS_Name','PDC','Jurisdiction','Segment_Length','CoSS_Congestion','CoSS_Reliability','CoSS_Rail_Reliability','CoSS_Capacity_Preservation','CoSS_LA_TDM','CoSS_non_LA_TDM','CoSS_Safety_Intersection','CoSS_Safety_Segments','RN_Congestion','RN_Reliability','RN_Capacity_Preservation','RN_LA_TDM','RN_non_LA_TDM','RN_AC_Bicycle_Access','RN_AC_Pedestrian_Access','RN_AC_Transit_Access','RN_Transit_Equity','Safety_Segments','Safety_Intersection','Safety_Pedestrian','IEDA']
df_needs = pd.DataFrame([row for row in arcpy.da.SearchCursor(needs_source, needs_fields)], columns=needs_fields)

# Load GeoID Lookup Tables
lookup_excel = 'Data_Explorer_Update.xlsx'
geoIDs_Districts = pd.read_excel(lookup_excel, sheet_name='District_IDs')
geoIDs_PDCs = pd.read_excel(lookup_excel, sheet_name='PDC_IDs')
geoIDs_Jurisdictions = pd.read_excel(lookup_excel, sheet_name='Jurisdiction_IDs')

In [ ]:
# Set up output DataFrame
df_output_fields = ['statId','statistic','travelMarket','direction','unit','unitDescription','relativePriority','value','geoId','year']
df_output = pd.DataFrame(columns=df_output_fields)

### 10037 - At least One Need Present (Directional Miles) ###
Calculated for Statewide, District, PDC, and Jurisdiction

In [ ]:
data = []

# Statewide
data.append({
    'geoId': 51000,
    'value': df_needs['Segment_Length'].sum()
})

# District, PDC, and Jurisdiction
def get_10037_data(geography, geoID_Dict):
    df_10037 = df_needs.groupby(geography)['Segment_Length'].sum().reset_index()
    df_10037 = df_10037.merge(geoID_Dict, left_on=geography, right_on='Name')
    df_10037.rename(columns={'GeoID':'geoId', 'Segment_Length': 'value'}, inplace=True)
    
    return df_10037

df_district_totals = get_10037_data('VDOT_District', geoIDs_Districts)
data.extend(df_district_totals[['geoId', 'value']].to_dict('records'))
# District totals will be referenced later

df_pdc_totals = get_10037_data('PDC', geoIDs_PDCs)
data.extend(df_pdc_totals[['geoId', 'value']].to_dict('records'))
del(df_pdc_totals)

df_jurisdiction_totals = get_10037_data('Jurisdiction', geoIDs_Jurisdictions)
data.extend(df_jurisdiction_totals[['geoId', 'value']].to_dict('records'))
del(df_jurisdiction_totals)


# Fill in missing attributes
statId = 10037
statistic = 'At least One Need Present (Directional Miles)'
travelMarket = 'CoSS, RN, UDA, Safety'
direction = 'N.A'
unit = 'Number'
unitDescription = 'Directional Miles'
relativePriority = 'Need Present'
year = 2023
for record in data:
    record['statId'] = statId
    record['statistic'] = statistic
    record['travelMarket'] = travelMarket
    record['direction'] = direction
    record['unit'] = unit
    record['unitDescription'] = unitDescription
    record['relativePriority'] = relativePriority
    record['year'] = year

# Add data to output DataFrame
df_output = df_output.append(data)

# Check count of records.  Previous 
print('10037 - At least One Need Present (Directional Miles)')
print('Previous count: 164')
print(f'Current count: {len(data)}')
print(f'Difference: {abs(164-len(data))}')

10037 - At least One Need Present (Directional Miles)
Previous count: 164
Current count: 156
Difference: 8


### 10039 - At least One Need Present (Share of District Total) ###
Previously these were calculated for jurisdictions only

In [ ]:
# Data to append to output DataFrame
data = []


# Make dictionary of district total mileage
df_district_totals['value'] = round(df_district_totals['value'], 3)
dict_district_totals = df_district_totals[['Name', 'value']].set_index('Name').to_dict()['value']
# del(df_district_totals)

def get_share_of_district(row):
    district = row['VDOT_District']
    district_total = dict_district_totals[district]
    share = round(row['value'] / district_total, 3)
    row['value'] = share
    
    return row

df_10039 = df_needs.groupby(['Jurisdiction', 'VDOT_District'])['Segment_Length'].sum().reset_index()
df_10039 = df_10039.merge(geoIDs_Jurisdictions, left_on='Jurisdiction', right_on='Name')
df_10039.rename(columns={'GeoID':'geoId', 'Segment_Length': 'value'}, inplace=True)
df_10039 = df_10039.apply(get_share_of_district, axis=1)
data.extend(df_10039[['geoId', 'value']].to_dict('records'))


# Fill in missing attributes
statId = 10039
statistic = 'At least One Need Present (Share of District Total)'
travelMarket = 'CoSS, RN, UDA, Safety'
direction = 'N.A'
unit = 'Percent'
unitDescription = 'Share of District Total'
relativePriority = 'Need Present'
year = 2023
for record in data:
    record['statId'] = statId
    record['statistic'] = statistic
    record['travelMarket'] = travelMarket
    record['direction'] = direction
    record['unit'] = unit
    record['unitDescription'] = unitDescription
    record['relativePriority'] = relativePriority
    record['year'] = year


# Add data to output DataFrame
df_output = df_output.append(data)

# Check count of records.  Previous 
print('10039 - At least One Need Present (Share of District Total)')
print('Previous count: 133')
print(f'Current count: {len(data)}')
print(f'Difference: {abs(164-len(data))}')

10039 - At least One Need Present (Share of District Total)
Previous count: 133
Current count: 140
Difference: 24


### 10040 - At least One Need Present (Share of Statewide Total) ###
Previously these were calculated for jurisdictions only

In [ ]:
# Data to append to output DataFrame
data = []


# Find statewide total
statewide_total = df_needs['Segment_Length'].sum()

def get_share_of_statewide(row):
    share = round(row['value'] / statewide_total, 3)
    row['value'] = share
    
    return row

df_10040 = df_needs.groupby('Jurisdiction')['Segment_Length'].sum().reset_index()
df_10040 = df_10040.merge(geoIDs_Jurisdictions, left_on='Jurisdiction', right_on='Name')
df_10040.rename(columns={'GeoID':'geoId', 'Segment_Length': 'value'}, inplace=True)
df_10039 = df_10040.apply(get_share_of_statewide, axis=1)
data.extend(df_10040[['geoId', 'value']].to_dict('records'))


# Fill in missing attributes
statId = 10040
statistic = 'At least One Need Present (Share of Statewide Total)'
travelMarket = 'CoSS, RN, UDA, Safety'
direction = 'N.A'
unit = 'Percent'
unitDescription = 'Share of District Total'
relativePriority = 'Need Present'
year = 2023
for record in data:
    record['statId'] = statId
    record['statistic'] = statistic
    record['travelMarket'] = travelMarket
    record['direction'] = direction
    record['unit'] = unit
    record['unitDescription'] = unitDescription
    record['relativePriority'] = relativePriority
    record['year'] = year


# Add data to output DataFrame
df_output = df_output.append(data)

# Check count of records.  Previous 
print('10039 - At least One Need Present (Share of District Total)')
print('Previous count: 133')
print(f'Current count: {len(data)}')
print(f'Difference: {abs(164-len(data))}')

10039 - At least One Need Present (Share of District Total)
Previous count: 133
Current count: 130
Difference: 34
